In [16]:
#get the sheet from the workbook
import numpy as np
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
loc="/Users/samuelsmith/Desktop/Python/ExcelFiles/TexasAirways.xlsx"
texasair=load_workbook(loc)
sheet=texasair["Data"]

In [17]:
#get the starting column and starting cells from the user
row=input("Starting cell for row labels:",)


#import python module for alphabet
import string 
alph=set(string.ascii_uppercase)

#split the input into two parts..letter and number
letter=row[0]
numeric=row.replace(row[0],"")
print(numeric)

#validate that the user entered a capital letter followed by a number 
if set(letter).issubset(alph) and numeric.isdigit()==True:
    introw=int(numeric)
#find the list of names for the row headers
    test="test"
    mylist=[]
    while type(sheet[row[0]+str(introw)].value)==type(test):
        mylist=mylist+[(sheet[row[0]+str(introw)].value)]
        introw=introw+1
    size=len(mylist)
    print("Possible cities are: ",mylist)
else:
    print("User input must be a valid capital letter followed by a number!")




Starting cell for row labels:D2
2
Possible cities are:  ['Dallas', 'Houston', 'Austin', 'Total Distance', 'San Antonio', 'Lubbock', 'El Paso']


In [18]:
#if there is data entered....perform 
if size>=1:
    letters=list(string.ascii_uppercase)
    #create the lists of lists of data
    bgncol=letters.index(row[0])+2
    Biglist=[]

    #extract the data from the worksheet
    for k in range(0,size):
        newlist=[]
        for i in range(0,size):
            if type(sheet.cell(row=int(numeric)+k,column=bgncol+i).value)==type(2):
                newlist.insert(i,sheet.cell(row=int(numeric)+k,column=int(bgncol)+i).value)
                
            else:
                newlist.insert(i,0) 
        Biglist=Biglist+[newlist]
    
    #Create the symmetric D matrix without using for loops
    D=np.array(Biglist)
    D=np.maximum(D,D.T)
    D[D==0]=1000
    np.fill_diagonal(D,0)
    print("Distance Matrix=","\n",D)
    Dprime=D.copy()#creates a copy of origingal D matrix
else:
    print("Sorry, but we can't proceed without any data")



Distance Matrix= 
 [[   0   55   40 1000 1000 1000 1000]
 [  55    0   35   60 1000 1000 1000]
 [  40   35    0 1000   30 1000 1000]
 [1000   60 1000    0   35 1000  100]
 [1000 1000   30   35    0   80 1000]
 [1000 1000 1000 1000   80    0   75]
 [1000 1000 1000  100 1000   75    0]]


In [19]:
if size>=1:
    #create the P matrix
    #remove the 1000 and replace with -1
    P=D.copy()
    print("Original P matrix","\n",P)
    P[P>=1000]=-1


    #deal with issue of arc and column values without a loop
    find=np.where(P>0)
    x=list(find[0])
    y=list(find[1])
    P[x,y]=y

    #fill in the diagonal
    rng=np.arange(0,size)
    np.fill_diagonal(P,rng)
    print("Desired P matrix","\n",P)

    Pprime=P.copy()
else:
    print("Sorry, but we can't proceed without any data")


Original P matrix 
 [[   0   55   40 1000 1000 1000 1000]
 [  55    0   35   60 1000 1000 1000]
 [  40   35    0 1000   30 1000 1000]
 [1000   60 1000    0   35 1000  100]
 [1000 1000   30   35    0   80 1000]
 [1000 1000 1000 1000   80    0   75]
 [1000 1000 1000  100 1000   75    0]]
Desired P matrix 
 [[ 0  1  2 -1 -1 -1 -1]
 [ 0  1  2  3 -1 -1 -1]
 [ 0  1  2 -1  4 -1 -1]
 [-1  1 -1  3  4 -1  6]
 [-1 -1  2  3  4  5 -1]
 [-1 -1 -1 -1  4  5  6]
 [-1 -1 -1  3 -1  5  6]]


In [20]:
if size>=1:
    #create the iterations for the D matrix
    for k in range(0,size):
        for i in range(0,size):
            for j in range(0,size):
                if Dprime[i,j]<=Dprime[i,k]+Dprime[k,j]:
                    Dprime[i,j]=Dprime[i,j]
                    Pprime[i,j]=Pprime[i,j]
                else:
                    Dprime[i,j]=Dprime[i,k]+Dprime[k,j]
                    Pprime[i,j]=Pprime[i,k]
    print("The final D matrix ","\n",Dprime)
    print("The final P matrix ","\n",Pprime)



The final D matrix  
 [[  0  55  40 105  70 150 205]
 [ 55   0  35  60  65 145 160]
 [ 40  35   0  65  30 110 165]
 [105  60  65   0  35 115 100]
 [ 70  65  30  35   0  80 135]
 [150 145 110 115  80   0  75]
 [205 160 165 100 135  75   0]]
The final P matrix  
 [[0 1 2 2 2 2 2]
 [0 1 2 3 2 2 3]
 [0 1 2 4 4 4 4]
 [4 1 4 3 4 4 6]
 [2 2 2 3 4 5 3]
 [4 4 4 4 4 5 6]
 [3 3 3 3 3 5 6]]


In [21]:
if size>=1:
    def CalculateTrip(Dfinal,Pfinal,cities):
        #save header for tables in excel
        sheet["A"+str(size+2+int(numeric))].value="Starting flight"
        sheet["B"+str(size+2+int(numeric))].value="Destination"
        sheet["C"+str(size+2+int(numeric))].value="Total Distance"
        sheet["D"+str(size+2+int(numeric))].value="Sequence of flights"
        
        texasair.save(loc)
        
        #create variable to move output location
        iterations=0
        
        #get input from user
        cont=input("Proceed with finding the distance and path? ",)
        
        #if they dont want to continue, stop the program
        if cont!="yes" and cont!="Yes":
            TotalDistance="You chose not to continue"
            finalpath=""
            return TotalDistance,finalpath
        
        #else, run the program for computations
        while cont=="yes"or cont=="Yes":
            iterations+=1
            import numpy as np
            trip=input("starting destination,(insert space after comma) final destination ",).split(", ")
            
            if len(trip)==2:
                
                #data validation that city is an input
                test=set(trip).issubset(set(cities))

                if test==False:
                    TotalDistance="none"
                    finalpath="Error! The entry of cities is not correct. Please review the guidelines."
                elif trip[0]==trip[1]:
                    TotalDistance="none"
                    finalpath="Starting and ending destination must be different cities!"

                if test==True and trip[0]!=trip[1]:
                    start=cities.index(trip[0])
                    end=cities.index(trip[1])
                    TotalDistance=Dfinal[start,end]

                    path=[start]
                    while start!=end:
                        start=Pprime[start,end]
                        path=path+[start]
                    finalpath=list(np.array(cities)[path])

                #convert list to string and remove 
                strpath=str(finalpath).replace("[","")
                strpath2=strpath.replace("]","")
                strpath3=strpath2.replace("'","")

                #output to excel
                sheet["A"+str(size+2+int(numeric)+iterations)].value=trip[0]
                sheet["B"+str(size+2+int(numeric)+iterations)].value=trip[1]
                sheet["C"+str(size+2+int(numeric)+iterations)].value=TotalDistance
                sheet["D"+str(size+2+int(numeric)+iterations)].value=strpath3
                
                
                #save to file
                texasair.save(loc)


                #print stuff for Python user
                print("Total Distance is: ", TotalDistance,"\n","The path is: ",finalpath)

                #ask user if they want to continue
                cont=input("Unsatisfied with results and want to try again? ",)

                #if cont=no, then stop
                if cont!="yes" and cont!="Yes":
                    return TotalDistance,finalpath
            else:
                return "You must input a starting and ending destination",""
            
    #return the final distance and path to the calling variable
    distance,how=CalculateTrip(Dprime,Pprime,mylist)
    print("The Results:","\n","Here is total distance: ", distance,".","Here is the path:" ,how)

Proceed with finding the distance and path? yes
starting destination,(insert space after comma) final destination no
The Results: 
 Here is total distance:  You must input a starting and ending destination . Here is the path: 
